<a href="https://colab.research.google.com/github/Prithvishankarmenon/NLP_Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# 📌 Step 1: Install Required Libraries
!pip install newspaper3k scikit-learn nltk networkx joblib
!pip install lxml_html_clean
!pip install newspaper

  Using cached newspaper-0.1.0.7.tar.gz (176 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [22]:
# 📌 Step 2: Import All Libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from joblib import dump, load
from newspaper import Article
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [23]:
# 📌 Step 3: Preprocessing
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = str(text).lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)


In [24]:
# 📌 Step 4: Load GNews Dataset
# 📍 Make sure to upload the dataset (gnews.csv) with "text" and "category" columns.
df = pd.read_csv("/content/drive/MyDrive/Data/train.csv")
# Rename for convenience
df.columns = ['class_id', 'title', 'description']

# Map class IDs to labels
category_map = {
    1: 'World',
    2: 'Sports',
    3: 'Business',
    4: 'Sci/Tech'
}
df['category'] = df['class_id'].map(category_map)

# Combine title and description
df['text'] = df['title'].fillna('') + ". " + df['description'].fillna('')
df['cleaned'] = df['text'].apply(clean_text)

df[['category', 'text']].head()


,category,text
0,Business,Wall St. Bears Claw Back Into the Black (Reute...
1,Business,Carlyle Looks Toward Commercial Aerospace (Reu...
2,Business,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,Business,Iraq Halts Oil Exports from Main Southern Pipe...
4,Business,"Oil prices soar to all-time record, posing new..."


In [26]:
df.head()

,class_id,title,description,category,text,cleaned
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business,Wall St. Bears Claw Back Into the Black (Reute...,wall bear claw back black reuter reuter wall s...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business,Carlyle Looks Toward Commercial Aerospace (Reu...,carlyl look toward commerci aerospac reuter re...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business,Oil and Economy Cloud Stocks' Outlook (Reuters...,oil economi cloud stock outlook reuter reuter ...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business,Iraq Halts Oil Exports from Main Southern Pipe...,iraq halt oil export main southern pipelin reu...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business,"Oil prices soar to all-time record, posing new...",oil price soar record pose new menac us econom...


In [27]:
# 📌 Step 5: Train a Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(df['cleaned'], df['category'], test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('nb', MultinomialNB())
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))
dump(pipeline, "news_classifier.joblib")


              precision    recall  f1-score   support

    Business       0.85      0.86      0.85      5911
    Sci/Tech       0.88      0.86      0.87      6075
      Sports       0.94      0.97      0.96      6058
       World       0.91      0.88      0.90      5956

    accuracy                           0.89     24000
   macro avg       0.89      0.89      0.89     24000
weighted avg       0.89      0.89      0.89     24000



['news_classifier.joblib']

In [45]:
def summarize(text, top_n=2, max_words=40):
    sentences = sent_tokenize(text)
    if len(sentences) <= top_n:
        return ' '.join(sentences[:top_n])

    tfidf = TfidfVectorizer().fit_transform(sentences)
    sim_matrix = cosine_similarity(tfidf)
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    # Sort by score and sentence position to preserve natural order
    ranked = sorted(((scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)

    summary = []
    total_words = 0

    for _, sentence, idx in sorted(ranked[:len(sentences)], key=lambda x: x[2]):
        word_count = len(sentence.split())
        if total_words + word_count <= max_words:
            summary.append(sentence)
            total_words += word_count
        if len(summary) >= top_n or total_words >= max_words:
            break

    return ' '.join(summary)


In [46]:
# 📌 Step 7: Scraper + Inference Pipeline
def fetch_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.title, article.text

def infer_news_category_and_summary(url):
    clf = load("news_classifier.joblib")
    title, text = fetch_article(url)
    cleaned = clean_text(title + ". " + text)
    category = clf.predict([cleaned])[0]
    summary = summarize(text)
    return {
        "title": title,
        "category": category,
        "summary": summary
    }


In [47]:
# 📌 Step 8: Run the Pipeline on Live News URL
url = "https://www.thehindu.com/sci-tech/science/axiom-4-mission-isro-coordinating-with-axiom-space-to-refresh-time-sensitive-experimental-specimens/article69701001.ece"
result = infer_news_category_and_summary(url)

print("📰 Title:\n", result['title'])
print("\n📂 Predicted Category:", result['category'])
print("\n📝 Summary:\n", result['summary'])


📰 Title:
 Axiom-4 mission: ISRO coordinating with Axiom Space to refresh time-sensitive experimental specimens

📂 Predicted Category: Sci/Tech

📝 Summary:
 With the launch of Indian astronaut Group Captain Shubhanshu Shukla’s mission to the International Space Station (ISS) scheduled for June 19, the Indian Space Research Organisation (ISRO) is coordinating with Axiom Space to refresh time-sensitive experimental specimens.
